In [64]:
import pandas as pd
import tensorflowjs as tfjs
import keras
import numpy as np
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Dropout, Flatten
from keras.utils import to_categorical
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

In [65]:
keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)

In [222]:
df = pd.read_csv("../data/training-candles.csv")

In [223]:
df.head()

,mts,open,close,high,low,volume,pair,timeframe,upDownThreeMethods,upsideGapTwoCrows,...,minus_dm,plus_di,minus_di,cvi,adx,apo,rocr,percentChange,strategy,strategyCode
0,1526498100000,0.68349,0.68284,0.68349,0.68243,56191.016894,tXRPUSD,15m,0,0,...,0.229636,0.318071,0.200967,0.051767,0.197311,0.540099,1.003188,-0.065901,LONG,1
1,1526499000000,0.68288,0.68321,0.68897,0.68288,236040.759933,tXRPUSD,15m,0,0,...,0.206240,0.457381,0.172625,0.108033,0.227721,0.550229,1.010427,0.054156,SHORT,0
2,1526499900000,0.68326,0.68100,0.68471,0.68100,133033.100087,tXRPUSD,15m,0,0,...,0.254890,0.425244,0.226975,0.084969,0.240472,0.544298,1.002458,-0.324523,SHORT,0
3,1526500800000,0.68087,0.67855,0.68156,0.67800,103130.321829,tXRPUSD,15m,0,0,...,0.341990,0.395461,0.322991,0.076800,0.230279,0.524175,1.000826,-0.361064,LONG,1
4,1526501700000,0.67855,0.67891,0.67989,0.67848,23766.613050,tXRPUSD,15m,0,0,...,0.310569,0.383687,0.313179,0.025578,0.220813,0.510663,0.996273,0.053026,LONG,1


In [224]:
def create_model(optimizer="adam", init_mode="uniform", activation="sigmoid"):
    model = Sequential()

    model.add(Dense(48, kernel_initializer=init_mode, input_shape=(46,), activation=activation))
    model.add(Dropout(0.2))
    model.add(Dense(24, kernel_initializer=init_mode, activation=activation))
    model.add(Dropout(0.2))
    model.add(Dense(12, kernel_initializer=init_mode, activation=activation))
    model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer=init_mode, activation="sigmoid"))

    model.compile(loss="binary_crossentropy",
                  optimizer=optimizer,
                  metrics=["accuracy"])
    return model

In [225]:
X = df[["adx", "cci", "rocr", "hikkake", "harami", "closingMarubozu", 
        "onNeck", "longLineCandle", "hikkakeModified", "apo", "cmo", "adx",
        "cvi", "plus_dm", "minus_dm", "plus_di", "minus_di", "dpo", "kvo",
        "fosc", "fisher", "fisher_signal", "dx", "linregslope", "macd", "macd_signal", 
        "macd_histogram", "mfi", "mom", "obv", "ppo", "pvi", "rsi", "stoch_k", "stoch_d",
        "atr", "trix", "ultosc", "vosc", "willr", "cmo", "cci", "adosc", "adxr", "ao",
        "aroonosc"
       ]].values

y = df["strategyCode"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [209]:
model = KerasClassifier(build_fn=create_model, verbose=0)

batch_size = [128]
epochs = [100]
optimizer = ['Adadelta', 'Adam']
init_mode = ['normal', 'uniform']
activation = ['relu', 'softplus', 'sigmoid']

param_grid = dict(batch_size=batch_size, epochs=epochs, optimizer=optimizer, init_mode=init_mode, activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

In [210]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.556526 using {'activation': 'softplus', 'batch_size': 128, 'epochs': 100, 'init_mode': 'normal', 'optimizer': 'Adam'}
0.537538 (0.006668) with: {'activation': 'relu', 'batch_size': 128, 'epochs': 100, 'init_mode': 'normal', 'optimizer': 'Adadelta'}
0.537320 (0.011068) with: {'activation': 'relu', 'batch_size': 128, 'epochs': 100, 'init_mode': 'normal', 'optimizer': 'Adam'}
0.539284 (0.002561) with: {'activation': 'relu', 'batch_size': 128, 'epochs': 100, 'init_mode': 'uniform', 'optimizer': 'Adadelta'}
0.551506 (0.005115) with: {'activation': 'relu', 'batch_size': 128, 'epochs': 100, 'init_mode': 'uniform', 'optimizer': 'Adam'}
0.542994 (0.011447) with: {'activation': 'softplus', 'batch_size': 128, 'epochs': 100, 'init_mode': 'normal', 'optimizer': 'Adadelta'}
0.556526 (0.014336) with: {'activation': 'softplus', 'batch_size': 128, 'epochs': 100, 'init_mode': 'normal', 'optimizer': 'Adam'}
0.545832 (0.004886) with: {'activation': 'softplus', 'batch_size': 128, 'epochs': 100, 'in

In [226]:
optimal_model = create_model(optimizer="Adam", init_mode="normal", activation="softplus")

#y_labels = to_categorical(y_train, num_classes=2)

optimal_model.fit(X_train, y_train, epochs=250, batch_size=128, verbose=1)

Epoch 1/250
4582/4582 [==============================] - 8s 2ms/step - loss: 0.6940 - acc: 0.4963
Epoch 2/250
4582/4582 [==============================] - 0s 32us/step - loss: 0.6938 - acc: 0.5022
Epoch 3/250
4582/4582 [==============================] - 0s 34us/step - loss: 0.6931 - acc: 0.5094
Epoch 4/250
4582/4582 [==============================] - 0s 31us/step - loss: 0.6930 - acc: 0.5052
Epoch 5/250
4582/4582 [==============================] - 0s 29us/step - loss: 0.6927 - acc: 0.5140
Epoch 6/250
4582/4582 [==============================] - 0s 28us/step - loss: 0.6923 - acc: 0.5129
Epoch 7/250
4582/4582 [==============================] - 0s 29us/step - loss: 0.6908 - acc: 0.5266
Epoch 8/250
4582/4582 [==============================] - 0s 31us/step - loss: 0.6901 - acc: 0.5325
Epoch 9/250
4582/4582 [==============================] - 0s 34us/step - loss: 0.6899 - acc: 0.5382
Epoch 10/250
4582/4582 [==============================] - 0s 29us/step - loss: 0.6890 - acc: 0.5375
Epoch 11/2

In [227]:
#y_test_labels = to_categorical(y_test, num_classes=2)
evaluation = optimal_model.evaluate(x=X_test, y=y_test, steps=1000)
optimal_model.metrics_names
print(evaluation)

1000/1000 [==============================] - 7s 7ms/step
[0.7376618981361389, 0.5172719359397888]


In [228]:
tfjs.converters.save_keras_model(optimal_model, "../public/model/")

In [214]:
optimal_model.predict_classes(X_test)

array([[0],
       [0],
       [0],
       ...,
       [1],
       [0],
       [0]], dtype=int32)